In [1]:
import sys
sys.path.append('../')

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import akshare as ak
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from database.downloader.downloader_base import DownloaderBase
import database.database_config as db_config

pd.options.display.max_rows=None
pd.options.display.max_columns=None

In [2]:
def plot_series_dist(series):
    data = series
    # 使用matplotlib画直方图
    plt.hist(data, bins=60, edgecolor='k', alpha=0.7)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Histogram of Data')
    plt.show()

class PreProcessing:
    def __init__(self, db_downloader:DownloaderBase) -> None:
        self.db_downloader = db_downloader

    def _build_label(self, stock_dataframe):
        N = 5 # 最大持仓周期 = N天，第N+1天开盘卖出
        df = stock_dataframe.copy()
        # 标签构建
        df['future_return'] = df['close'].shift(-N) / df['open'].shift(-1) - 1 # 计算第N日收益率
        # 极值处理
        df['future_return'] = np.clip(
            df['future_return'], 
            np.nanquantile(df['future_return'], 0.01), 
            np.nanquantile(df['future_return'], 0.99),
            )
        # 过滤第二天一字涨停情况
        df = df[df['high'].shift(-1) != df['low'].shift(-1)]
        return df[['datetime', 'future_return']]


    def _process_one_stock(self, stock_code, start_date, end_date):
        stock_base = self.db_downloader._download_stock_base_info(stock_code) # 获取基础代码
        stock_individual = self.db_downloader._download_stock_individual_info(stock_code) # 获取profile信息
        stock_history = self.db_downloader._download_stock_history_info(stock_code, start_date, end_date) # 获取历史行情
        stock_indicator = self.db_downloader._download_stock_indicator_info(stock_code, start_date, end_date) # 获取指标数据
        stock_factor_date = self.db_downloader._download_stock_factor_date_info() # 获取日期特征
        stock_factor_qlib = self.db_downloader._download_stock_factor_qlib_info(stock_code, start_date, end_date) # 获取量价特征
        stock_label = self._build_label(stock_history) # 构建label
        stock_df = stock_base.merge(stock_individual, on=['stock_code']).merge(stock_history, on=['stock_code']).merge(stock_indicator, on=['stock_code', 'datetime']).merge(stock_label, on=['datetime']).merge(stock_factor_date, on=['datetime']).merge(stock_factor_qlib, on=['stock_code', 'datetime']) # 整合数据
        stock_df = stock_base \
            .merge(stock_individual, on=['stock_code', 'stock_name']) \
            .merge(stock_history, on=['stock_code']) \
            .merge(stock_indicator, on=['stock_code', 'datetime']) \
            .merge(stock_label, on=['datetime']) \
            .merge(stock_factor_date, on=['datetime']) \
            .merge(stock_factor_qlib, on=['stock_code', 'datetime']) # 整合数据
        stock_df = stock_df.dropna()
        return stock_df
    
    def _process_all_stock(self, code_type, start_date, end_date):
        # stock_code_list = list(ak.stock_info_a_code_name()['code'].unique()) # 获取A股所有股票列表
        # stock_code_list = list(ak.index_stock_cons("000905")['品种代码'].unique()) # 获取中证500的股票代码列表
        stock_code_list = list(ak.index_stock_cons(code_type)['品种代码'].unique()) # 获取沪深300的股票代码列表
        # stock_code_list = list(ak.index_stock_cons(code_type)['品种代码'].unique()) # 获取中证50的股票代码列表
        stock_df_list = []
        for stock_code in tqdm(stock_code_list, desc=f'Process: {code_type} ...'):
            stock_df = self._process_one_stock(stock_code, start_date, end_date)
            if not stock_df.empty:
                stock_df_list.append(stock_df)
        return pd.concat(stock_df_list)
        # return stock_df_list

In [3]:
db_conn = sqlite3.connect('../database/hh_quant.db')
db_downloader = DownloaderBase(db_conn, db_config)
proprocessor = PreProcessing(db_downloader=db_downloader)

## 使用Tensorflow

In [4]:
# 使用tensorflow处理原始数据
import numpy as np
import pandas as pd
import tensorflow as tf
from model import QuantModel
print(tf.__version__)

2024-03-18 17:19:44.138655: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.15.0


In [5]:
def extract_train_val_data(df, train_start_date, train_end_date, val_start_date, val_end_date):
    train_start_date = pd.to_datetime(train_start_date)
    train_end_date = pd.to_datetime(train_end_date)
    val_start_date = pd.to_datetime(val_start_date)
    val_end_date = pd.to_datetime(val_end_date)

    train_data = df[(pd.to_datetime(df['datetime']) >= train_start_date) & (pd.to_datetime(df['datetime']) <= train_end_date)]
    val_data = df[(pd.to_datetime(df['datetime']) >= val_start_date) & (pd.to_datetime(df['datetime']) <= val_end_date)]

    print(f"train_data_size: {train_data.shape}")
    print(f"validation_data_size: {val_data.shape}")
    return train_data, val_data

def transfer_data_type(df, columns, dtype):
    for col in columns:
        df[col] = df[col].astype(dtype)
    return df

def get_numeric_boundaries(series, num_bins=30):
    if series.nunique() < num_bins:
        boundaries = sorted(series.unique())
    else:
        boundaries = pd.qcut(series, num_bins, retbins=True, duplicates='drop')[1].tolist()
    return boundaries

def df_to_dataset(dataframe, feature_cols, label_cols, shuffle=True, batch_size=32):
    features = dataframe[feature_cols]
    labels = [dataframe[label_col] for label_col in label_cols]
    ds = tf.data.Dataset.from_tensor_slices((dict(features), tuple(labels)))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(features))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [6]:
backtest_start_date = '20200101'
backtest_end_date = '20240101'
train_period = 6 # year：训练数据周期长度
update_period = 6 # month：模型更新周期长度

def get_rolling_date_period(backtest_start_date, backtest_end_date, training_period, update_period):
    backtest_start_date = datetime.strptime(backtest_start_date, '%Y%m%d')
    backtest_end_date = datetime.strptime(backtest_end_date, '%Y%m%d')
    result = []
    rolling_flag = True
    while rolling_flag:
        current_val_start_date = backtest_start_date
        current_val_end_date = current_val_start_date + relativedelta(months=update_period) - relativedelta(days=1)
        if current_val_start_date < backtest_end_date:
            current_train_start_date = current_val_start_date - relativedelta(years=training_period)
            current_train_end_date = current_val_start_date - relativedelta(days=1)
            result.append([
                current_train_start_date.strftime("%Y%m%d"),
                current_train_end_date.strftime("%Y%m%d"),
                current_val_start_date.strftime("%Y%m%d"),
                current_val_end_date.strftime("%Y%m%d")
                ])
            backtest_start_date += relativedelta(months=update_period) 
        else:
            rolling_flag=False # 结束滚动训练
    return result

rolling_period = get_rolling_date_period(backtest_start_date, backtest_end_date, train_period, update_period)
rolling_period

[['20140101', '20191231', '20200101', '20200630'],
 ['20140701', '20200630', '20200701', '20201231'],
 ['20150101', '20201231', '20210101', '20210630'],
 ['20150701', '20210630', '20210701', '20211231'],
 ['20160101', '20211231', '20220101', '20220630'],
 ['20160701', '20220630', '20220701', '20221231'],
 ['20170101', '20221231', '20230101', '20230630'],
 ['20170701', '20230630', '20230701', '20231231']]

In [7]:
# df = proprocessor._process_all_stock(code_type='000300', start_date='20170701', end_date='20231231')

In [8]:
feature_config = {
    "target_feature_name": ["future_return"],
    "numeric_features": ['turnover_rate', 'pe_ttm', 'ps_ttm', 'pcf_ncf_ttm', 'pb_mrq', 'KMID', 'KLEN', 'KMID2', 'KUP', 'KUP2', 'KLOW', 'KLOW2', 'KSFT', 'KSFT2', 'OPEN0', 'OPEN1', 'OPEN2', 'OPEN3', 'OPEN4', 'HIGH0', 'HIGH1', 'HIGH2', 'HIGH3', 'HIGH4', 'LOW0', 'LOW1', 'LOW2', 'LOW3', 'LOW4', 'CLOSE0', 'CLOSE1', 'CLOSE2', 'CLOSE3', 'CLOSE4', 'VOLUME0', 'VOLUME1', 'VOLUME2', 'VOLUME3', 'VOLUME4', 'ROC5', 'ROC10', 'ROC20', 'ROC30', 'ROC60', 'MAX5', 'MAX10', 'MAX20', 'MAX30', 'MAX60', 'MIN5', 'MIN10', 'MIN20', 'MIN30', 'MIN60', 'MA5', 'MA10', 'MA20', 'MA30', 'MA60', 'STD5', 'STD10', 'STD20', 'STD30', 'STD60', 'BETA5', 'BETA10', 'BETA20', 'BETA30', 'BETA60', 'RSQR5', 'RSQR10', 'RSQR20', 'RSQR30', 'RSQR60', 'RESI5', 'RESI10', 'RESI20', 'RESI30', 'RESI60', 'QTLU5', 'QTLU10', 'QTLU20', 'QTLU30', 'QTLU60', 'QTLD5', 'QTLD10', 'QTLD20', 'QTLD30', 'QTLD60', 'TSRANK5', 'TSRANK10', 'TSRANK20', 'TSRANK30', 'TSRANK60', 'RSV5', 'RSV10', 'RSV20', 'RSV30', 'RSV60', 'IMAX5', 'IMAX10', 'IMAX20', 'IMAX30', 'IMAX60', 'IMIN5', 'IMIN10', 'IMIN20', 'IMIN30', 'IMIN60', 'IMXD5', 'IMXD10', 'IMXD20', 'IMXD30', 'IMXD60', 'CORR5', 'CORR10', 'CORR20', 'CORR30', 'CORR60', 'CORD5', 'CORD10', 'CORD20', 'CORD30', 'CORD60', 'CNTP5', 'CNTP10', 'CNTP20', 'CNTP30', 'CNTP60', 'CNTN5', 'CNTN10', 'CNTN20', 'CNTN30', 'CNTN60', 'CNTD5', 'CNTD10', 'CNTD20', 'CNTD30', 'CNTD60', 'SUMP5', 'SUMP10', 'SUMP20', 'SUMP30', 'SUMP60', 'SUMN5', 'SUMN10', 'SUMN20', 'SUMN30', 'SUMN60', 'SUMD5', 'SUMD10', 'SUMD20', 'SUMD30', 'SUMD60', 'VMA5', 'VMA10', 'VMA20', 'VMA30', 'VMA60', 'VSTD5', 'VSTD10', 'VSTD20', 'VSTD30', 'VSTD60', 'WVMA5', 'WVMA10', 'WVMA20', 'WVMA30', 'WVMA60', 'VSUMP5', 'VSUMP10', 'VSUMP20', 'VSUMP30', 'VSUMP60', 'VSUMN5', 'VSUMN10', 'VSUMN20', 'VSUMN30', 'VSUMN60', 'VSUMD5', 'VSUMD10', 'VSUMD20', 'VSUMD30', 'VSUMD60'],
    "integer_categorical_features": ['weekday', 'day_of_month', 'month'],
    "string_categorical_features": ['industry', 'day_of_week', 'season']
}
full_feature_names = feature_config.get('numeric_features', []) + feature_config.get('integer_categorical_features', []) + feature_config.get('string_categorical_features', [])
stock_pool = '000300' # [上证50:000016, 沪深300:000300, 中证500:000905]所有股票作为训练数据
batch_size = 256

for date_period in tqdm(rolling_period, desc='Rolling Training...'):
    train_start_date, train_end_date, val_start_date, val_end_date = date_period
    print(f"train_start: {train_start_date}, train_end: {train_end_date}, val_start: {val_start_date}, val_end: {val_end_date}")
    # 1. 获取所有股票信息
    df = proprocessor._process_all_stock(code_type=stock_pool, start_date=train_start_date, end_date=val_end_date)
    # 2. 拆分训练数据&验证数据
    train_data, val_data = extract_train_val_data(df, train_start_date, train_end_date, val_start_date, val_end_date)
    # 3. 构建训练集和验证集
    train_ds = df_to_dataset(train_data, full_feature_names, feature_config.get('target_feature_name', []), shuffle=True, batch_size=batch_size)
    val_ds = df_to_dataset(val_data, full_feature_names, feature_config.get('target_feature_name', []), shuffle=False, batch_size=batch_size)
    # 4. 配置模型相关参数
    model_config = {
        "seed": 1024,
        "reduction_ratio": 3,
        "cross_layer_num": 2,
        "parameterization": 'matrix', # vector, matrix
        "dnn_hidden_units": [64,32],
        "dnn_activation": 'relu',
        "dnn_dropout": 0.2,
        "dnn_use_bn": True,
        "numeric_features_with_boundaries": {k: list(get_numeric_boundaries(train_data[k])) for k in feature_config.get('numeric_features', [])},
        "integer_categorical_features_with_vocab": {k: list(train_data[k].unique()) for k in feature_config.get('integer_categorical_features', [])},
        "string_categorical_features_with_vocab": {k: list(train_data[k].unique()) for k in feature_config.get('string_categorical_features', [])},
        "feature_embedding_dims": 6,
        "task_type": ['reg'],
    }
    # 5. 初始化模型
    model = QuantModel(model_config)
    # 6. 配置训练相关参数
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        verbose=2,
        patience=10,
        mode='min',
        restore_best_weights=True,
    )
    # 7. 配置Tensorboard记录功能
    log_dir = "./logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # 8. 配置optimizer
    initial_learning_rate = 5e-4
    # lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    #     initial_learning_rate,
    #     decay_steps=1000,
    #     decay_rate=0.8,
    #     staircase=True)
    # optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=initial_learning_rate) # for Mac M1/M2
    optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate) # for intel
    loss = [tf.keras.losses.MeanSquaredError()]
    model.compile(optimizer=optimizer, loss=loss)
    model.fit(
            train_ds, 
            validation_data=val_ds, 
            epochs=50,
            verbose=2,
            callbacks=[tensorboard_callback, early_stopping]
    )
    # 9. 配置保存模型功能
    # model_save_path = f'./models/saved_model/model_of_{val_start_date}'
    # model.save(model_save_path)
    # best_model = tf.keras.models.load_model('./best_model')

    # 10. 记录预测集合
    model_red_result = model.predict(val_ds)
    output_df = val_data[['stock_code', 'stock_name', 'datetime']]
    output_df['future_return'] = val_data['future_return']
    output_df['future_return_pred'] = model_red_result
    output_file_path = f'../../Offline/backtest/backtest_data/stock_selection_results_{val_start_date}.pkl'
    output_df.to_pickle(output_file_path)

Rolling Training...:   0%|          | 0/8 [00:00<?, ?it/s]

train_start: 20140101, train_end: 20191231, val_start: 20200101, val_end: 20200630


Process: 000300 ...: 100%|██████████| 284/284 [00:43<00:00,  6.46it/s]


train_data_size: (267459, 207)
validation_data_size: (27100, 207)
CrossNet parameterization: vector
Epoch 1/50


2024-03-18 17:34:03.639264: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 106847 of 267459
2024-03-18 17:34:13.638095: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 217663 of 267459
2024-03-18 17:34:18.675005: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


...

1181/1181 - 73s - loss: 0.0032 - val_loss: 0.0039 - 73s/epoch - 62ms/step
Epoch 6/50


2024-03-18 19:21:26.819814: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:28: Filling up shuffle buffer (this may take a while): 128660 of 302083
2024-03-18 19:21:36.819792: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:28: Filling up shuffle buffer (this may take a while): 249359 of 302083
2024-03-18 19:21:41.318090: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


1181/1181 - 64s - loss: 0.0032 - val_loss: 0.0039 - 64s/epoch - 54ms/step
Epoch 7/50


2024-03-18 19:22:31.038120: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:28: Filling up shuffle buffer (this may take a while): 128644 of 302083
2024-03-18 19:22:41.038101: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:28: Filling up shuffle buffer (this may take a while): 248749 of 302083
2024-03-18 19:22:45.733823: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
Rolling Training...:  38%|███▊      | 3/8 [2:03:34<3:25:57, 2471.56s/it]


KeyboardInterrupt: 

In [ ]:
# model.summary()

In [9]:
model_config

{'seed': 1024,
 'reduction_ratio': 3,
 'cross_layer_num': 1,
 'dnn_hidden_units': ListWrapper([64, 32]),
 'dnn_activation': 'relu',
 'dnn_dropout': 0.1,
 'dnn_use_bn': True,
 'numeric_features_with_boundaries': {'turnover_rate': ListWrapper([0.0044, 0.0822, 0.133, 0.1787, 0.2217, 0.2642, 0.3065, 0.3491, 0.393, 0.4388, 0.4863, 0.535, 0.5876, 0.6435, 0.7049, 0.7695, 0.8422, 0.9216, 1.0108, 1.1115599999999977, 1.2295, 1.3673, 1.5272, 1.7216, 1.9582, 2.267, 2.6735400000000022, 3.256379999999999, 4.165019999999996, 5.977059999999986, 48.0089]),
  'pe_ttm': ListWrapper([-13343.970644, -12.4208332, 5.751325, 6.6932036, 7.7512612, 9.078301, 10.5933854, 12.052986, 13.362868599999999, 14.774811199999998, 16.125140999999996, 17.7369758, 19.21677, 20.810897, 22.519195399999994, 24.517737, 26.620819399999995, 29.0810812, 31.6705172, 34.48067299999999, 37.561443999999995, 41.066705799999994, 45.152682999999996, 50.289314799999985, 57.262058600000024, 66.76766500000002, 77.95702500000002, 92.99423219